# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets

In [2]:
from pyomo import environ as pe
from modules.global_set import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,5)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[5].T_F
| reactive[5].P
| reactive[5].cat
| reactive[5].Q_main
| reactive[5].x_
| reactive[5].y_
| reactive[5].x
| reactive[5].y
| reactive[5].z
| reactive[5].L
| reactive[5].V
| reactive[5].F
| reactive[5].H_L_
| reactive[5].H_V_
| reactive[5].H_L
| reactive[5].H_V
| reactive[5].T
| reactive[5].H_F
| reactive[5].f_V
| reactive[5].f_L
| reactive[5].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[5].kinetics_block.k_FT
| reactive[5].kinetics_block.r_FT_total
| reactive[5].kinetics_block.g0_FT
| reactive[5].kinetics_block.alpha
| reactive[5].kinetics_block.r_FT_cnum
| reactive[5].kinetics_block.r_FT_comp
| reactive[5].kinetics_block.k_WGS
| reactive[5].kinetics_block.Ke_WGS
| reactive[5].kinetics_block.r_WGS
| reactive[5].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Linking Stage Variables

### Vapor Between Reactive Stages

In [6]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [7]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [8]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [9]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

In [10]:
model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

# Load from single stage solutions

In [11]:
with open('../saved_solutions/1_stage_condenser_adiabatic.pickle', 'rb') as f:
    results_imported = pickle.load(f)
results_changed = results_imported

### Duplicate variable solution and bounds multiplier

In [12]:
for i in list(results_imported.Solution.Variable.keys()):
    if i.startswith('reactive[1].'):
        for j in model.TRAY:
            if j != 1:
                results_changed.Solution.Variable[i.replace('reactive[1].','reactive[{}].'.format(j))] = \
                results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [13]:
for i in list(results_changed.Solution.Constraint.keys()):
    if i.startswith('reactive[1].'):
        for j in model.TRAY:
            if j != 1:
                results_changed.Solution.Constraint[i.replace('reactive[1].','reactive[{}].'.format(j))] = \
                results_imported.Solution.Constraint[i]

### Load changed solution into current model

In [14]:
model.solutions.load_from(results_changed)

# Fixing Redundent Stream Variables

In [15]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [16]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

# Load Operating Parameters

In [25]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2))
    model.reactive[j].z['H2'].fix(2/(1+2))
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    # model.reactive[j].T.setub(240+273.15)
model.reactive[1].T.setub(210+273.15)
model.reactive[2].T.setub(215+273.15)
model.reactive[3].T.setub(220+273.15)
model.reactive[4].T.setub(225+273.15)
model.reactive[5].T.setub(230+273.15)

In [18]:
check_DOF(pe,model)

Active Equality Constraints:	 7672
Active Inequality Constraints:	 0
Active Variables:		 7899
Fixed Variables:		 222
DOF:				 5


In [19]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [20]:
add_dual(pe,model)
update_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [26]:
results = opt.solve(model,tee=True)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma86                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:    25272
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

  74 -2.4655735e+03 2.34e+01 1.55e+03  -1.0 1.18e+04    -  5.37e-05 6.21e-04h  1
  75 -2.4655832e+03 2.34e+01 1.17e+04  -1.0 1.53e+04    -  4.27e-03 3.48e-04h  1
  76 -2.4656116e+03 2.34e+01 2.09e+04  -1.0 1.61e+04    -  3.99e-03 1.26e-03h  1
  77 -2.4654504e+03 2.33e+01 5.76e+03  -1.0 1.48e+04    -  3.12e-05 4.56e-03h  1
  78 -2.4654404e+03 2.33e+01 6.61e+03  -1.0 2.17e+04    -  4.06e-04 1.47e-04h  1
  79 -2.4653073e+03 2.32e+01 3.95e+03  -1.0 1.67e+04    -  4.18e-05 1.83e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -2.4653134e+03 2.32e+01 3.95e+03  -1.0 1.37e+04    -  6.77e-04 4.47e-05h  1
  81 -2.4654250e+03 2.32e+01 3.77e+03  -1.0 1.68e+04    -  1.29e-04 8.22e-04f  1
  82 -2.4654467e+03 2.32e+01 3.76e+03  -1.0 1.60e+04    -  7.61e-04 2.00e-04h  1
  83 -2.4654362e+03 2.32e+01 1.17e+04  -1.0 7.53e+03    -  3.08e-03 2.26e-04h  1
  84 -2.4653950e+03 2.32e+01 4.28e+04  -1.0 1.13e+04    -  1.04e-02 8.64e-04h  1
  85 -2.4653940e+03 2.32e+01

 167 -2.4206736e+03 4.67e+00 3.22e+03  -1.0 2.66e+01    -  3.80e-02 1.40e-01h  3
 168 -2.4210676e+03 4.96e+00 2.79e+05  -1.0 3.63e+01    -  1.13e-01 6.11e-01H  1
 169 -2.4229496e+03 9.48e+00 4.88e+05  -1.0 1.21e+02    -  7.82e-01 9.86e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170 -2.4276932e+03 1.92e+00 1.11e+06  -1.0 2.94e+02    -  3.53e-01 9.92e-01H  1
 171 -2.4357184e+03 4.53e-01 1.91e+06  -1.0 5.31e+02    -  7.46e-01 9.81e-01H  1
 172 -2.4561147e+03 1.86e+00 5.73e+06  -1.0 1.64e+03    -  3.32e-01 1.00e+00F  1
 173 -2.4576688e+03 1.67e+00 6.90e+06  -1.0 1.10e+03    -  9.29e-01 9.91e-02f  3
 174 -2.4589523e+03 1.76e+00 6.73e+06  -1.0 1.56e+03    -  8.79e-01 6.29e-02f  3
 175 -2.4605200e+03 2.04e+00 5.89e+06  -1.0 1.20e+03    -  1.00e+00 1.30e-01f  3
 176 -2.4616814e+03 2.08e+00 5.05e+06  -1.0 8.96e+02    -  1.00e+00 1.45e-01f  3
 177 -2.4651378e+03 4.00e+00 2.14e+06  -1.0 6.81e+02    -  1.00e+00 5.77e-01f  1
 178 -2.4655322e+03 6.89e-02

In [27]:
update_dual(pe,model)

In [28]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               30.00 	       -72.2282175381 	         0.9434840101 	         0.0000000000
              210.00 	       -44.1733150170 	         2.2168633921 	         0.0109574191
              215.00 	       -45.2464065794 	         1.7500147043 	         0.0211914664
              220.00 	       -45.8479838263 	         1.2999674807 	         0.0300099794
              225.00 	       -46.1486473530 	         0.8607477848 	         0.0368347010
              230.00 	       -46.2802563742 	         0.4282762795 	         0.0412962553
----------------------------------------------------------------------------------------------------
Top
V	 0.943484010080068
L	 0.07114846320014526
W	 1.20223091882719
----------------------------------------------------------------------------------------------------
Bottom
L	 0.041296255264909765
-----------------

In [29]:
model.reactive[1].kinetics_block.r_FT_total.value

0.26651082350106553

In [24]:
# model.solutions.store_to(results)
# with open('../saved_solutions/5_stage_condenser_220_240C.pickle','wb') as f:
#     pickle.dump(results,f)

In [30]:
model.condenser.L['out'].fixed = False
for j in model.reactive:
    model.reactive[j].T.fixed=True

In [31]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.condenser.L['out'],sense=pe.maximize)

# So, what exactly does adding a reflux do?

In [32]:
Refluxrange = np.linspace(0,2,21)

In [33]:
cd_data = {};
cd_data['Re'] = []; cd_data['D'] = []; cd_data['V'] = []
cd_data['x'] = {}; cd_data['y'] = {}; cd_data['g'] = {}; cd_data['d'] = {};
for i in m.COMP_TOTAL:
    cd_data['x'][i] = []
    cd_data['y'][i] = []
    cd_data['g'][i] = []
    cd_data['d'][i] = []
    
rf_data = {}
for j in model.reactive:
    rf_data[j] = {}
    rf_data[j]['r'] = {}; rf_data[j]['b'] = {}; rf_data[j]['x'] = {};rf_data[j]['y'] = {};
    rf_data[j]['T'] = []; rf_data[j]['Q'] = []; rf_data[j]['V'] = []; rf_data[j]['L'] = []; 
    rf_data[j]['r_WGS'] = []; rf_data[j]['r_FT'] = []
    for i in m.COMP_TOTAL:
        rf_data[j]['r'][i] = []
        rf_data[j]['b'][i] = []
        rf_data[j]['x'][i] = []
        rf_data[j]['y'][i] = []       

In [34]:
for re in Refluxrange:
    # model.condenser.L['out'].fixed = False
    model.condenser.L['out'].setub(re)
    # model.reactive[1].T.fix(t)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
          format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reactive[3].L['out'].value))

    cd_data['V'].append(model.condenser.V['out'].value)
    cd_data['D'].append(model.condenser.L['P'].value)
    cd_data['Re'].append(model.condenser.L['out'].value)

    for i in model.reactive[1].r_total_comp:
        cd_data['x'][i].append(model.condenser.x[i].value)
        cd_data['y'][i].append(model.condenser.y[i].value)
        cd_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
        cd_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
    
    for j in model.reactive:      
        rf_data[j]['T'].append(model.reactive[j].T.value)
        rf_data[j]['Q'].append(model.reactive[j].Q_main.value)
        rf_data[j]['V'].append(model.reactive[j].V['out'].value)
        rf_data[j]['L'].append(model.reactive[j].L['out'].value)
        rf_data[j]['r_WGS'].append(model.reactive[j].kinetics_block.r_WGS.value)
        rf_data[j]['r_FT'].append(model.reactive[j].kinetics_block.r_FT_total.value)

        for i in model.reactive[1].r_total_comp:
            rf_data[j]['r'][i].append(model.reactive[j].r_total_comp[i].value)
            rf_data[j]['b'][i].append(model.reactive[j].x[i].value*model.reactive[j].L['out'].value)
            rf_data[j]['x'][i].append(model.reactive[j].x[i].value)
            rf_data[j]['y'][i].append(model.reactive[j].y[i].value)

Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.943 kmol/s	|Distillate = 0.071 kmol/s	|Bottom = 0.030 kmol/s
Solved	|Reflux = 0.100 kmol/s	|Vapor = 0.952 kmol/s	|Distillate = 0.066 kmol/s	|Bottom = 0.036 kmol/s
Solved	|Reflux = 0.200 kmol/s	|Vapor = 0.960 kmol/s	|Distillate = 0.061 kmol/s	|Bottom = 0.042 kmol/s
Solved	|Reflux = 0.300 kmol/s	|Vapor = 0.969 kmol/s	|Distillate = 0.057 kmol/s	|Bottom = 0.049 kmol/s
Solved	|Reflux = 0.400 kmol/s	|Vapor = 0.978 kmol/s	|Distillate = 0.054 kmol/s	|Bottom = 0.055 kmol/s
Solved	|Reflux = 0.500 kmol/s	|Vapor = 0.988 kmol/s	|Distillate = 0.051 kmol/s	|Bottom = 0.062 kmol/s
Solved	|Reflux = 0.600 kmol/s	|Vapor = 0.997 kmol/s	|Distillate = 0.048 kmol/s	|Bottom = 0.068 kmol/s
Solved	|Reflux = 0.700 kmol/s	|Vapor = 1.006 kmol/s	|Distillate = 0.045 kmol/s	|Bottom = 0.075 kmol/s
Solved	|Reflux = 0.800 kmol/s	|Vapor = 1.015 kmol/s	|Distillate = 0.043 kmol/s	|Bottom = 0.082 kmol/s
Solved	|Reflux = 0.900 kmol/s	|Vapor = 1.025 kmol/s	|Distillate = 0.041 kmol/s	|Bo

In [35]:
cnumber_range = range(1,57)

In [36]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in range(1,57):
            tmp[j].append(molefraction[i][j])
    return tmp

In [37]:
g_data = trans_cnumber(cd_data['g'])
d_data = trans_cnumber(cd_data['d'])
b_data = trans_cnumber(rf_data[3]['b'])
# r_data = trans_cnumber(rf_data_master[t]['r'])

cd_x_data = trans_cnumber(cd_data['x'])
rf_x_data = {}
for j in model.reactive:
    rf_x_data[j] = trans_cnumber(rf_data[j]['x'])

In [38]:
 def plot_distribution(index):
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(16,12))
    ax.plot(cnumber_range,g_data[index],'co-')
    ax.plot(cnumber_range,d_data[index],'go-')
    ax.plot(cnumber_range,b_data[index],'ro-')
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(cd_data['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)

    ax2.plot(cnumber_range,cd_x_data[index],'go-')
    ax2.plot(cnumber_range,rf_x_data[1][index],'co-')
    ax2.plot(cnumber_range,rf_x_data[2][index],'bo-')
    ax2.plot(cnumber_range,rf_x_data[3][index],'mo-')
    ax2.plot(cnumber_range,rf_x_data[4][index],'ko-')
    ax2.plot(cnumber_range,rf_x_data[5][index],'ro-')

    ax2.legend(['Condenser','Stage 1','Stage 2','Stage 3','Stage 4','Stage 5'],fontsize=18)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [39]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:',)
);

interactive(children=(IntSlider(value=0, description='Reflux:', max=20), Output()), _dom_classes=('widget-inte…